In [1]:
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import vgg16
from tqdm.notebook import tqdm
import os
import torch

In [2]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [3]:
def train_CNN(name,dataset_name):
    # Define data directory and image transforms
    data_dir = './drive/MyDrive/Colab Notebooks/Data/' #or data2, depending on which dataset you want to use
    # Create PyTorch datasets and dataloaders
    dataset = ImageFolder(os.path.join(data_dir, dataset_name), transform=data_transforms)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    # number of classes as number of directories in data_dir
    num_classes = len(os.listdir(data_dir + dataset_name))
    # Create VGG16 model with 5 neurons in last layer
    model = vgg16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    num_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_features, num_classes)
    # make progress bar using tqdm
    # tqdm._instances.clear()
    
    # Train the model
    i=0
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model.to(device)
    model = model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    num_epochs = 15
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            # print(len(outputs))
            # print(len(labels))
            # print(inputs.shape)
            # print(outputs.shape)
            # print(labels)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            i+=1
            if i%100==0:
                print('Iteration loss: {:.4f}'.format(loss.item()))
        epoch_loss = running_loss / len(dataset)
        print('Epoch {}/{} loss: {:.4f}'.format(epoch+1, num_epochs, epoch_loss))
    #save the model at path ../models/name.pth
    torch.save(model.state_dict(), './'+name+'.pth')
    torch.save(model.state_dict(), './drive/MyDrive/Colab Notebooks/'+ name + '.pth')
    print('Successfully trained ' + name + '!' )

In [ ]:
# train_CNN('White_Face_ID_CNN','White_Faces')
# train_CNN('Asian_Face_ID_CNN','Asian_Faces')
# train_CNN('Obj_Face_Cat_CNN','Obj-Face dataset')
# train_CNN('Cars_CNN','Cars')
# train_CNN('Face_ID_CNN_2','Faces')
# train_CNN('Face_ID_Inverted_CNN','Faces_Inverted')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 221MB/s]


cuda
